<a href="https://colab.research.google.com/github/sarunsmenon/llm/blob/main/telstra_support_crawler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Libraries

In [55]:
!pip install -q python-dotenv openai langchain-openai cohere langchain langchain_community pypdf faiss-gpu wikipedia-api faiss-cpu wikipedia langchainhub unstructured playwright uuid7

# Load Libraries

In [33]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import textwrap

In [34]:
import os
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv, find_dotenv
from google.colab import userdata
import pickle

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

In [35]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader, WikipediaLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.tools.retriever import create_retriever_tool

from langchain_community.document_loaders import UnstructuredURLLoader
from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor

from langchain import hub
from langchain_core.messages import AIMessage, HumanMessage
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

from uuid_extensions import uuid7str


# Load Variables

In [36]:
os.environ['OPENAI_API_KEY'] = userdata.get('open_ai_key')
prompt_template = "hwchase17/openai-functions-agent"

os.environ['LANGCHAIN_TRACING_V2']="true"
os.environ['LANGCHAIN_API_KEY']=userdata.get('langsmith_api_key')

session_id = uuid7str()

In [37]:
llm_model = 'gpt-3.5-turbo-1106'
llm = ChatOpenAI(model=llm_model, temperature=0)

In [38]:
# Start crawling from the initial URL
start_url = 'https://www.telstra.com.au/support'
ignore_lst = []
include_lst = ['support' ,'telstra']
max_pg_lmt = 5000
db_name = "faiss_telstra_support_db"
fldr = '/content/drive/MyDrive/Colab Notebooks/Langchain/telstra_support/'

In [39]:
if not os.path.exists(fldr):
    # If the folder does not exist, create it
    os.makedirs(fldr)
    print(f'Folder created at: {fldr}')
else:
    print(f'Folder already exists at: {fldr}')

Folder already exists at: /content/drive/MyDrive/Colab Notebooks/Langchain/telstra_support/


# Load Functions

In [40]:
# Function to get all links from a page
def get_all_links(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    links = [a.get('href') for a in soup.find_all('a', href=True)]
    full_links = [urljoin(url, link) for link in links]
    return full_links

In [41]:
print(max_pg_lmt)

5000


In [42]:
# Function to crawl the website
def crawl_website(start_url, max_pages=max_pg_lmt):
    itr = 0

    visited = set()
    to_visit = [start_url]

    while to_visit and len(visited) < max_pages:
      url = to_visit.pop(0)
      # if (
      #     # (
      #     #   any( [url for site in ignore_lst if site in url]) or
      #     #   (url in visited) or
      #     #   (not any ([url for site in include_lst if site in url]))
      #     # )
      #     ("telstra.com.au" not in url) and ("support" not in url) and (url not in visited)
      # ) :
      #     continue

      if (
          (url not in visited) and
          ("telstra.com.au" in url) and
          ("support" in url) and
          ("mobilesupport.telstra.com.au" not in url)
        ):
        visited.add(url)
        try:
          links = get_all_links(url)
          to_visit.extend(links)
        except:
          continue

        itr += 1
        if itr % 10 == 0:
          print(f"Visited {len(visited)}: {url}")

    return visited

In [43]:
def extract_process_url(url):
  loader = UnstructuredURLLoader(urls=[url])
  data = loader.load()

  text_splitter = CharacterTextSplitter(
                chunk_size=1000,
                chunk_overlap=5,
                separator= "\n\n",
                length_function=len,
                is_separator_regex=False
              )

  docs = text_splitter.split_documents(data)
  return docs


In [44]:
def store_doc_into_db(docs, faiss_rmit_db):
  if faiss_rmit_db is None:
    faiss_rmit_db = FAISS.from_documents(docs, OpenAIEmbeddings())
  else:
    faiss_rmit_db.add_documents(docs)

  return faiss_rmit_db


In [45]:
def generate_chat_response(message, local_session_id):
  result = agent_with_chat_history.invoke({"input": message}, config={"configurable": {"session_id": local_session_id}})
  return result['output']

# Create URL Lists

In [ ]:
visited_urls = crawl_website(start_url)
print(f"Total visited URLs: {len(visited_urls)}")

Visited 10: https://www.telstra.com.au/aboutus/support-in-times-of-need
Visited 20: https://www.telstra.com.au/support/account-payment/set-up-direct-debit
Visited 30: https://www.telstra.com.au/support/account-payment/give-ownership-of-account-to-someone
Visited 40: https://www.telstra.com.au/support/account-payment/order-estimate
Visited 50: https://www.telstra.com.au/support/internet-and-home-phone/connecting-to-wifi
Visited 60: https://www.telstra.com.au/support/internet-and-home-phone/set-up-and-manage-messagebank
Visited 70: https://www.telstra.com.au/support/internet-and-home-phone/caller-id-and-calling-number-display
Visited 80: https://www.telstra.com.au/support/internet-and-home-phone/report-a-fault
Visited 90: https://www.telstra.com.au/support/internet-and-home-phone/access-modem-settings
Visited 100: https://www.telstra.com.au/support/mobiles-devices/mobile-black-spot-program
Visited 110: https://www.telstra.com.au/support/pre-paid/extra-data
Visited 120: https://www.telstr

Visited 440: https://www.telstra.com.au/content/dam/tcom/personal/support/pdf/broadband/set-up-guide-for-opticomm-access-network-fttp.pdf


Visited 450: https://www.telstra.com.au/support/internet-and-home-phone/setup-telstra-smart-modem-for-adsl-hfc-cable#hfc-new-cable-internet


Visited 460: https://www.telstra.com.au/support/internet-and-home-phone/home-internet-order-next-steps#main-content
Visited 470: https://www.telstra.com.au/support/internet-and-home-phone/nbn-connection-types#lightbox-nbn-fw
Visited 480: https://www.telstra.com.au/support/internet-and-home-phone/setup-telstra-smart-modem#fttn-fttb
Visited 490: https://www.telstra.com.au/support/internet-and-home-phone/set-up-call-forwarding-on-home-phone#apply-call-waiting
Visited 500: https://www.telstra.com.au/support/internet-and-home-phone/prioritise-internet-optimiser-for-gaming-or-work#work


Visited 510: https://www.telstra.com.au/support/internet-and-home-phone/fix-internet-issues#main-content
Visited 520: https://www.telstra.com.au/support/internet-and-home-phone/technician-service-appointment#main-content
Visited 530: https://www.telstra.com.au/support/internet-and-home-phone/report-a-fault#after-reporting
Visited 540: https://www.telstra.com.au/support/internet-and-home-phone/smart-wi-fi-booster-support#compatibility
Visited 550: https://www.telstra.com.au/support/internet-and-home-phone/set-up-port-forwarding#main-content


...

Visited 1260: https://www.telstra.com.au/business-enterprise/support/tipt-phones#help-and-support


Visited 1270: https://www.telstra.com.au/content/dam/shared-component-assets/tecom/uc&c/collaboration/tipt/support/telstra-vvx-300-301-phone-guide.pdf


Visited 1280: https://www.telstra.com.au/content/dam/shared-component-assets/tecom/uc&c/collaboration/tipt/support/huawei-8950-quick-reference-guide.pdf


Visited 1290: https://www.telstra.com.au/business-enterprise/support/track-monitor#main-content
Visited 1300: https://www.telstra.com.au/business-enterprise/support/t-analyst/manage-your-invoices#schedule-an-email-invoice
Visited 1310: https://www.telstra.com.au/business-enterprise/support/t-analyst/reports#request
Visited 1320: https://www.telstra.com.au/business-enterprise/support/t-analyst/hierarchies#adding-service-descriptions


Visited 1330: https://www.telstra.com.au/business-enterprise/support/t-analyst/how-to-videos#main-content
Visited 1340: https://www.telstra.com.au/business-enterprise/support/t-analyst/how-to-videos#download-a-bill-T-Analyst
Visited 1350: https://www.telstra.com.au/business-enterprise/support/tipt/tipt-how-to-videos#summary
Total visited URLs: 1350


In [ ]:
with open(fldr+"url_list.pkl", "wb") as f:
    pickle.dump(visited_urls, f)

In [ ]:
with open(fldr+"url_list.pkl", "rb") as f:
    visited_urls = list(pickle.load(f))

error_url_lst = []

In [ ]:
print(f"Total visited URLs: {len(visited_urls)}")

Total visited URLs: 1350


# Create Database

In [ ]:
if os.path.exists(fldr+db_name):
  faiss_telstra_support_db = FAISS.load_local(fldr+db_name,embeddings=OpenAIEmbeddings(), allow_dangerous_deserialization=True)
  print("DB already exists")
else:
  faiss_telstra_support_db = None
  print("create a new database because none exists")


create a new database because none exists


In [ ]:
for url in visited_urls[0:5000]:
  print(url)
  try:
    docs = extract_process_url(url)
    faiss_telstra_support_db = store_doc_into_db(docs, faiss_telstra_support_db)
  except:
    error_url_lst.append(url)


https://www.telstra.com.au/business-enterprise/support/important-terms#other-services
https://www.telstra.com.au/support/mobiles-devices/device-security#device-security


https://www.telstra.com.au/support/internet-and-home-phone/setup-telstra-smart-modem-for-adsl-hfc-cable#set-up
https://www.telstra.com.au/support/entertainment/fetch#using-fetch
https://www.telstra.com.au/support/mobiles-devices#main-content
https://www.telstra.com.au/aboutus/support-in-times-of-need/adversity-financial-hardship
https://www.telstra.com.au/small-business/online-support/mobiles-devices/telstra-one-number#main-content
https://www.telstra.com.au/support/mobiles-devices/enable-volte-mobile-phone#tech-assist


https://www.telstra.com.au/content/dam/shared-component-assets/tecom/support/sip-connect/sip-connect-accredited-pbx-devices-summary.xlsx


https://www.telstra.com.au/content/dam/shared-component-assets/tecom/uc&c/collaboration/tipt/support/business-instantgroup-conferencing-quick-reference-guide.pdf


ERROR:langchain_community.document_loaders.url:Error fetching or processing https://www.telstra.com.au/content/dam/shared-component-assets/tecom/uc&c/collaboration/tipt/support/business-instantgroup-conferencing-quick-reference-guide.pdf, exception: partition_pdf is not available. Install the pdf dependencies with pip install "unstructured[pdf]"


...

https://www.telstra.com.au/small-business/online-support/mobiles-devices/change-mobile-plan#main-content
https://www.telstra.com.au/support/email/faq#set-up
https://www.telstra.com.au/support/mobiles-devices/activate-a-service-on-esim
https://www.telstra.com.au/support/pre-paid/fix-issue-with-pre-paid-service
https://www.telstra.com.au/support/internet-and-home-phone/5g-home-support#main-content
https://www.telstra.com.au/small-business/online-support/business-software/mcafee-multi-access-licence#main-content
https://www.telstra.com.au/content/dam/shared-component-assets/tecom/uc&c/collaboration/tipt/support/telstra-vvx-300-301-phone-guide.pdf


ERROR:langchain_community.document_loaders.url:Error fetching or processing https://www.telstra.com.au/content/dam/shared-component-assets/tecom/uc&c/collaboration/tipt/support/telstra-vvx-300-301-phone-guide.pdf, exception: partition_pdf is not available. Install the pdf dependencies with pip install "unstructured[pdf]"


https://www.telstra.com.au/support/account-payment/id-check-for-new-account#main-content


https://www.telstra.com.au/support/mobiles-devices/messagebank-call-forwarding-ring-time#messagebank-plus
https://www.telstra.com.au/business-enterprise/support/liberate#help
https://www.telstra.com.au/support/entertainment#main-content
https://www.telstra.com.au/business-enterprise/support/t-analyst/hierarchies#adding-service-descriptions
https://www.telstra.com.au/support/email/check-usage
https://www.telstra.com.au/support/internet-and-home-phone/smart-wi-fi-booster-support#main-content
https://www.telstra.com.au/support/mobiles-devices/device-security#set-up


https://www.telstra.com.au/small-business/online-support/business-software/raise-a-support-ticket#main-content
https://www.telstra.com.au/support/mobiles-devices/manage-esim#main-content
https://www.telstra.com.au/aboutus/support-in-times-of-need/disaster-relief
https://www.telstra.com.au/support/internet-and-home-phone/nbn-connection-types#main-content
https://www.telstra.com.au/small-business/online-support/business-software/set-up-office-365-domain-name#main-content
https://www.telstra.com.au/small-business/online-support/mobiles-devices/shared-data-allowance#main-content
https://www.telstra.com.au/small-business/online-support/business-software/telstra-apps-marketplace-terms-conditions
https://www.telstra.com.au/support/email
https://www.telstra.com.au/business-enterprise/support/important-terms#cloud-services
https://www.telstra.com.au/support/internet-and-home-phone/home-phone-connection-on-nbn-network#main-content
https://www.telstra.com.au/business-enterprise/support/changing-a

# Write Everything

In [ ]:
print(faiss_telstra_support_db)

In [ ]:
FAISS.save_local(faiss_telstra_support_db, fldr+db_name)

In [ ]:
with open(fldr+"error_urls.pkl", "wb") as f:
  pickle.dump(error_url_lst, f)

# Create Retriever

In [46]:
if os.path.exists(fldr+db_name):
  faiss_telstra_support_db = FAISS.load_local(fldr+db_name,embeddings=OpenAIEmbeddings(), allow_dangerous_deserialization=True)
else:
  faiss_telstra_support_db = None

In [47]:
primary_retriever = faiss_telstra_support_db.as_retriever( search_type="similarity_score_threshold",
                            search_kwargs={"score_threshold": 0.5,"k": 1}
)

In [48]:
primary_retriever.invoke("modem not working")

[Document(metadata={'source': 'https://www.telstra.com.au/support/internet-and-home-phone/check-smart-modem#main-content'}, page_content='When the modem boots up (usually 2-3 minutes) the mobile signal light on the top of the modem should light up which indicates the modem has successfully contacted the mobile network.\n\nStep 7: Not working? Contact us\n\nIf the mobile signal light does not light up or is red, contact us or message us through the My Telstra app.\n\nFrequently asked questions\n\nIf you have followed the step by step instructions above, and your mobile signal light is still off, please message us through the My Telstra app. One of our agents will help you.\n\nThere are currently three different generations of the Telstra Smart Modem. If you have any of these models (look on the back or base of your modem to find the model number), then you have a Telstra Smart Modem.\n\nThe Telstra Smart Modem 2.0 has ports arranged vertically down the back\n\nThe Telstra Smart Modem 1.

# Agents

In [49]:
telsta_support_tool = create_retriever_tool(
    primary_retriever,
    "telstra_support_search",
    "Search for support from Telstra"
)

In [50]:
tools = [telsta_support_tool ]

In [51]:
# Get the prompt to use - you can modify this!
prompt = hub.pull(prompt_template)
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [52]:
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False)

# Chat History

In [53]:
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    lambda session_id: ChatMessageHistory(),
    input_messages_key="input",
    history_messages_key="chat_history",
)

# Execution

In [54]:
input_qn  = "Hi Bot, I need help with Telstra Stuff"
while input_qn != "exit":
  # print(f" Input qn is : {input_qn}")
  print("")
  response = generate_chat_response(input_qn, session_id)
  print(textwrap.fill(response, 80))
  print("")
  input_qn = input("Enter a message (to finish use exit): ")



You can reach Telstra support by messaging them or calling 13 20 00. They are
available Monday to Friday from 8am to 7pm AEST. If you need further assistance,
feel free to ask!

Enter a message (to finish use exit): modem red light

The red light on your modem could indicate a few different issues, depending on
the type of modem you have. Here are some possibilities:  1. Telstra Smart Modem
Gen 2:    - If the light at the front of the modem is red, it may indicate a
connection issue with your normal home internet or the Telstra Mobile Network.
2. Telstra Frontier Gateway & Telstra Smart Modem:    - If the "online" light on
top of the modem is red, it may indicate a connection issue with your normal
home internet or the Telstra Mobile Network.  To troubleshoot this issue
further, it's important to check the specific model of your modem and refer to
the user manual for troubleshooting steps. If you need further assistance,
please let me know!

Enter a message (to finish use exit): iphon

# Testing

In [29]:
string = 'The white light on your Telstra Smart Modem indicates that it is starting up or powering up. If the light remains white for an extended period, it may be in the process of connecting to the internet. You can also check the status of your modem by entering http://192.168.0.1 in your browser and signing in using the default credentials. If you need further assistance, please let me know!'
# print("{string}:<40")
print(textwrap.fill(string, 80))


The white light on your Telstra Smart Modem indicates that it is starting up or
powering up. If the light remains white for an extended period, it may be in the
process of connecting to the internet. You can also check the status of your
modem by entering http://192.168.0.1 in your browser and signing in using the
default credentials. If you need further assistance, please let me know!


In [ ]:
from langchain_community.document_loaders import PlaywrightURLLoader

urls = [
    "https://www.youtube.com/watch?v=dQw4w9WgXcQ",
    "https://goo.gl/maps/NDSHwePEyaHMFGwh8",
]

loader = PlaywrightURLLoader(urls=visited_urls, remove_selectors=["header", "footer"])

data = await loader.aload()


ERROR:langchain_community.document_loaders.url_playwright:Error fetching or processing https://www.rmit.edu.au/staff, exception: Locator.all: Execution context was destroyed, most likely because of a navigation


CancelledError: 

In [ ]:
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
import requests

# Initialize embeddings and FAISS vector store
embeddings = OpenAIEmbeddings()
faiss_store = FAISS(embedding_function=embeddings)

# Function to fetch content from a URL and add it to the FAISS vector store
def fetch_and_add_to_faiss(url, faiss_store):
    try:
        response = requests.get(url)
        response.raise_for_status()
        text = response.text
        faiss_store.add_texts([text])
        print(f"Added content from {url} to FAISS store.")
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")



TypeError: FAISS.__init__() missing 3 required positional arguments: 'index', 'docstore', and 'index_to_docstore_id'

In [ ]:
# Process each URL and add to the FAISS store
for url in visited_urls:
    fetch_and_add_to_faiss(url, faiss_store)

print(f"FAISS vector store now contains {faiss_store.index.ntotal} vectors.")

# Save the FAISS vector store
faiss_store.save("faiss_store.pkl")


In [ ]:
with open(fldr+"visited_urls.pkl", "wb") as f:
    pickle.dump(visited_urls, f)

In [ ]:

with open(fldr+"visited_urls.pkl", "rb") as f:
    visited_urls = pickle.load(f)